<a href="https://colab.research.google.com/github/harenlin/PySpark-Learning/blob/main/Manipulating_Data_in_DataFrames_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MDD').getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

     |████████████████████████████████| 212.4MB 68kB/s 
     |████████████████████████████████| 204kB 18.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=6db09441fff5fff3a104c2ae3dd521308c8913b3b4f7c0a974d7c49171033e8a
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
You are working with 1 core(s)


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/PySpark/Datasets/'
tweets = spark.read.csv(path + 'Rep_vs_Dem_tweets.csv', inferSchema=True, header=True)

Mounted at /content/drive


## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [5]:
tweets.show(5)

+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         null|                null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+
only showing top 5 rows



**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [6]:
tweets.show(5,False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [8]:
tweets.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [11]:
from pyspark.sql.functions import *
latino = tweets.withColumn('Latino_Mentions', regexp_extract(tweets.Tweet, '(.)(@LatinoLeader)(.)', 0))
# https://blog.csdn.net/hqr20627/article/details/78038649
latino.show(5,False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Party                             |Handle       |Tweet                                                                                                                                       |Latino_Mentions|
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |               |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for Nati

In [12]:
from pyspark.sql.functions import *
latino = tweets.withColumn('Latino_Mentions', regexp_extract(tweets.Tweet, '(.)(@LatinoLeader)(.)', 1))
latino.show(5,False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Party                             |Handle       |Tweet                                                                                                                                       |Latino_Mentions|
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |               |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for Nati

In [13]:
from pyspark.sql.functions import *
latino = tweets.withColumn('Latino_Mentions', regexp_extract(tweets.Tweet, '(.)(@LatinoLeader)(.)', 2))
latino.show(5,False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Party                             |Handle       |Tweet                                                                                                                                       |Latino_Mentions|
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |               |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for Nati

## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [15]:
from pyspark.sql.functions import *
counts = tweets.groupBy("Party").count()
counts.orderBy(desc("count")).show(20,False)

+-----------------------------------------+-----+
|Party                                    |count|
+-----------------------------------------+-----+
|Republican                               |44392|
|Democrat                                 |42068|
|That’s…"                                 |28   |
|https://t.co/oc6JNAF5K5                  |22   |
|Now                                      |17   |
|Today                                    |13   |
|http…"                                   |12   |
|h…"                                      |12   |
|https://t…"                              |12   |
|❌ Terminated #DACA                       |11   |
|❌ Abandoned Hispanic outreach for #ACA   |11   |
|#mepolitics"                             |11   |
|❌ Passed #TaxScam                        |11   |
|❌ E…"                                    |11   |
|https://t.co…"                           |9    |
|https://t.co/8htzynw0mp"                 |9    |
|💻 Website: https://t.co/dqyGOuzN…"      |9    |
|

In [17]:
from pyspark.sql.functions import when
cleaned = tweets.withColumn('Party', when(tweets.Party == 'Democrat', 'Democrat').when(tweets.Party == 'Republican', 'Republican').otherwise('Other'))
cleaned_counts = cleaned.groupBy('Party').count()
cleaned_counts.orderBy(desc('count')).show()

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [19]:
# just glimpse the data
tweets.select("tweet").show(5,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|null                                                                                                                                        |

In [21]:
tweets.withColumn('cleaned_tweet', 
                  regexp_replace('Tweet', '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '')).select(['tweet', 'cleaned_tweet']).show(5,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|tweet                                                                                                                                       |cleaned_tweet                                                                                                                               |
+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |Today,

## 4. Remove any leading or trailing white space in the tweet column

In [24]:
from pyspark.sql.functions import *
tweets.select('Tweet', trim(tweets.Tweet)).show(5,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |trim(Tweet)                                                                                                                                 |
+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |Today,

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [28]:
renamed1 = tweets.withColumnRenamed('Party', 'Dem_Rep')
renamed1.show(5)

renamed2 = tweets.withColumn('Dem_Rep', tweets.Party)
renamed2.show(5)

# see the diff ?!

+--------------------+-------------+--------------------+
|             Dem_Rep|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         null|                null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+
only showing top 5 rows

+--------------------+-------------+--------------------+--------------------+
|               Party|       Handle|               Tweet|             Dem_Rep|
+--------------------+-------------+--------------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|            Democrat|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|            Democrat|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...| 

## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [29]:
from pyspark.sql.functions import *
tweets.select(tweets.Party, tweets.Handle, concat_ws(' ', tweets.Party, tweets.Handle).alias('Concatenated')).show(5,False)

+----------------------------------+-------------+----------------------------------+
|Party                             |Handle       |Concatenated                      |
+----------------------------------+-------------+----------------------------------+
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
|Congress has allocated about $18…"|null         |Congress has allocated about $18…"|
|Democrat                          |RepDarrenSoto|Democrat RepDarrenSoto            |
+----------------------------------+-------------+----------------------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [30]:
# THIS IS FROM HW SOLUTION

from pyspark.sql.functions import *
# Parenthesis are used to mark a subexpression within a larger expression
# The . matches any character other than a new line
# | means is like or
# \w+ means followed by any word
pattern = '(.|'')(#)(\w+)'
# * is used to match the preceding character zero or more times.
# ? will match the preceding character zero or one times, but no more.
# $ is used to match the ending position in a string. 
split_pattern = r'.*?({pattern})'.format(pattern=pattern)
end_pattern = r'(.*{pattern}).*?$'.format(pattern=pattern)

# $1 here means to capture the first part of the regex result
# The , will separate each find with a comma in the a array we create
df2 = tweets.withColumn('a', regexp_replace('Tweet', split_pattern, '$1,')).where(col('Tweet').like('%#%'))
df2.select('a').show(3,False)
# Remove all the other results that came up
df3 = df2.withColumn('a', regexp_replace('a', end_pattern, '$1'))
df3.select('a').show(3,False)
# Finally create an array from the result by splitting on the comma
df4 = df3.withColumn('a', split('a', r','))
df4.select('a').show(3,False)
df4.limit(3).toPandas()

+-----------------------------------------------------------------------------------------+
|a                                                                                        |
+-----------------------------------------------------------------------------------------+
| #SaveTheInternet, #NetNeutrality, legislation here in the House… https://t.co/n3tggDLU1L|
| #NALCABPolicy2018,.…                                                                    |
| #NetNeutrality, rules. Find out…                                                        |
+-----------------------------------------------------------------------------------------+
only showing top 3 rows

+---------------------------------+
|a                                |
+---------------------------------+
| #SaveTheInternet, #NetNeutrality|
| #NALCABPolicy2018               |
| #NetNeutrality                  |
+---------------------------------+
only showing top 3 rows

+------------------------------------+
|a             

,Party,Handle,Tweet,a
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...","[ #SaveTheInternet, #NetNeutrality]"
1,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[ #NALCABPolicy2018]
2,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSo...,[ #NetNeutrality]


# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [31]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office, ['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [32]:
df = df.withColumn("dob", df["dob"].cast(DateType())) \
       .withColumn("visit1", df["visit1"].cast(DateType())) \
       .withColumn("visit2", df["visit2"].cast(DateType())) \
       .withColumn("visit3", df["visit3"].cast(DateType()))

df.show()
print(df.printSchema())

+----------+---------+----------+----------+----------+----------+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|
+----------+---------+----------+----------+----------+----------+
|  Mohammed|   Alfasy|1987-04-08|2016-01-07|2017-02-03|2018-03-02|
|     Marcy|Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|
|     Ginny|   Ginger|1986-07-10|2014-08-07|2015-02-03|2016-03-02|
|     Vijay| Doberson|1988-05-02|2016-01-07|2018-02-03|2018-03-02|
|     Orhan|  Gelicek|1987-05-11|2016-05-07|2017-01-03|2018-09-02|
|     Sarah|    Jones|1956-07-06|2016-04-07|2017-08-03|2018-10-02|
|      John|  Johnson|2017-10-12|2018-01-02|2018-10-03|2018-03-02|
+----------+---------+----------+----------+----------+----------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

None


## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [35]:
from pyspark.sql.functions import *
diff1 = df.select(datediff(df.visit2, df.visit1).alias("diff"))
diff2 = df.select(datediff(df.visit3, df.visit2).alias("diff"))

diff1.show(5)
diff2.show(5)

+----+
|diff|
+----+
| 393|
| 727|
| 180|
| 758|
| 241|
+----+
only showing top 5 rows

+----+
|diff|
+----+
| 392|
| 364|
| 393|
|  27|
| 607|
+----+
only showing top 5 rows



In [34]:
diff_combo = diff1.union(diff2)
diff_combo.show(5,False)

+----+
|diff|
+----+
|393 |
|727 |
|180 |
|758 |
|241 |
+----+
only showing top 5 rows



## 8. Can you calculate the age of each patient?

In [40]:
# dob = date of birth
ages = df.select(format_number( datediff( df.visit1, df.dob)/365, 2).alias("age"))
ages.show()

+-----+
|  age|
+-----+
|28.77|
|28.77|
|28.10|
|27.70|
|29.01|
|59.79|
| 0.22|
+-----+



In [43]:
ages2 = df.withColumn("age", format_number( datediff( df.visit1, df.dob)/365, 2))
ages2.show(5,False)

+----------+---------+----------+----------+----------+----------+-----+
|first_name|last_name|dob       |visit1    |visit2    |visit3    |age  |
+----------+---------+----------+----------+----------+----------+-----+
|Mohammed  |Alfasy   |1987-04-08|2016-01-07|2017-02-03|2018-03-02|28.77|
|Marcy     |Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|28.77|
|Ginny     |Ginger   |1986-07-10|2014-08-07|2015-02-03|2016-03-02|28.10|
|Vijay     |Doberson |1988-05-02|2016-01-07|2018-02-03|2018-03-02|27.70|
|Orhan     |Gelicek  |1987-05-11|2016-05-07|2017-01-03|2018-09-02|29.01|
+----------+---------+----------+----------+----------+----------+-----+
only showing top 5 rows



## 9. Can you extract the month from the first visit column and call it "Month"?

In [44]:
month = df.withColumn("Month", month(df.visit1))
month.show(5,False)

+----------+---------+----------+----------+----------+----------+-----+
|first_name|last_name|dob       |visit1    |visit2    |visit3    |Month|
+----------+---------+----------+----------+----------+----------+-----+
|Mohammed  |Alfasy   |1987-04-08|2016-01-07|2017-02-03|2018-03-02|1    |
|Marcy     |Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|1    |
|Ginny     |Ginger   |1986-07-10|2014-08-07|2015-02-03|2016-03-02|8    |
|Vijay     |Doberson |1988-05-02|2016-01-07|2018-02-03|2018-03-02|1    |
|Orhan     |Gelicek  |1987-05-11|2016-05-07|2017-01-03|2018-09-02|5    |
+----------+---------+----------+----------+----------+----------+-----+
only showing top 5 rows



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [63]:
path = '/content/drive/My Drive/PySpark/Datasets/'
sales = spark.read.csv(path + 'supermarket_sales.csv', inferSchema=True, header=True)

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [64]:
sales.show(5,False)

+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|Invoice ID |Branch|City     |Customer type|Gender|Product line          |Unit price|Quantity|Tax 5% |Total   |Date     |Time |Payment    |cogs  |gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|A     |Yangon   |Member       |Female|Health and beauty     |74.69     |7       |26.1415|548.9715|1/5/2019 |13:08|Ewallet    |522.83|4.761904762            |26.1415     |9.1   |
|226-31-3081|C     |Naypyitaw|Normal       |Female|Electronic accessories|15.28     |5       |3.82   |80.22   |3/8/2019 |10:29|Cash       |76.4  |4.761904762            |3.82        |9.6   |
|631-41-3108|A     |Yangon   |Normal       |M

In [65]:
sales.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [66]:
sales2 = sales.withColumn("Date", sales.Date.cast(DateType()))
print(sales2.printSchema())

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

None


In [67]:
sales2.select('Date').show(5)

+----+
|Date|
+----+
|null|
|null|
|null|
|null|
|null|
+----+
only showing top 5 rows



In [84]:
from pyspark.sql.types import *
# from pyspark.sql.functions import *

print("Note that this method gives all null values")
df = sales.withColumn("Formatted Date", sales["Date"].cast(DateType()))
df = df.select("Date", "Formatted Date")
print(df.limit(6).toPandas())

print(" ")
print("This result gives the wrong results (notice that all months are january)")
sales.select('Date', to_date(sales.Date, 'm/d/yyyy').alias('Dateformatted'), month(to_date(sales.Date, 'm/d/yyyy')).alias('Month')).show(5)

print(" ")
print("But if we capitalize the mm part in the format, we get the correct results!")
print("Props to your fellow classmate David Henderson for figuring this one out!")
sales.select('Date', to_date(sales.Date, 'M/d/yyyy').alias('Dateformatted'), month(to_date(sales.Date, 'M/d/yyyy')).alias('Month')).show(5)

Note that this method gives all null values
        Date Formatted Date
0   1/5/2019           None
1   3/8/2019           None
2   3/3/2019           None
3  1/27/2019           None
4   2/8/2019           None
5  3/25/2019           None
 
This result gives the wrong results (notice that all months are january)
+---------+-------------+-----+
|     Date|Dateformatted|Month|
+---------+-------------+-----+
| 1/5/2019|   2019-01-05|    1|
| 3/8/2019|   2019-01-08|    1|
| 3/3/2019|   2019-01-03|    1|
|1/27/2019|   2019-01-27|    1|
| 2/8/2019|   2019-01-08|    1|
+---------+-------------+-----+
only showing top 5 rows

 
But if we capitalize the mm part in the format, we get the correct results!
Props to your fellow classmate David Henderson for figuring this one out!
+---------+-------------+-----+
|     Date|Dateformatted|Month|
+---------+-------------+-----+
| 1/5/2019|   2019-01-05|    1|
| 3/8/2019|   2019-03-08|    3|
| 3/3/2019|   2019-03-03|    3|
|1/27/2019|   2019-01-27|   

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [73]:
from pyspark.sql.types import *
# split the date field and get the month value 
df = sales.select('Date', split('Date', '/')[0].alias('Month'), 'Total')

# Verify everything worked correctly
print("Verify")
df.show(5)
print(df.printSchema())

Note that previous method gives all null values (notice that all months are january)
 
Verify
+---------+-----+--------+
|     Date|Month|   Total|
+---------+-----+--------+
| 1/5/2019|    1|548.9715|
| 3/8/2019|    3|   80.22|
| 3/3/2019|    3|340.5255|
|1/27/2019|    1| 489.048|
| 2/8/2019|    2|634.3785|
+---------+-----+--------+
only showing top 5 rows

root
 |-- Date: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Total: double (nullable = true)

None


## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [74]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [76]:
# We can do 1-3 in one call here
df = reviews.withColumn("cleaned_reviews", trim(lower(regexp_replace(col('review_txt'),'[^\sa-zA-Z0-9]', ''))))
df.show(5,False)

+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                            |cleaned_reviews                                                                    |
+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |epic this is the best movie i have ever seen                                       |
|4     |Pretty good, but I would have liked to seen better special effects                    |pretty good but i would have liked to seen better special effects                  |
|3     |So so. Casting could have been improved                                               |so so

In [77]:
# Then split on the spaces!
df = df.withColumn("review_txt_array", split(col("cleaned_reviews"), " "))
df.show(5,False)

+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |cleaned_reviews                                                                    |review_txt_array                                                                                   |
+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |epic this is the best movie i have ever seen                     

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [78]:
epic = df.withColumn("result", array_contains(col("review_txt_array"), "epic"))
epic.toPandas()

,rating,review_txt,cleaned_reviews,review_txt_array,result
0,5,Epic. This is the best movie I have EVER seen,epic this is the best movie i have ever seen,"[epic, this, is, the, best, movie, i, have, ev...",True
1,4,"Pretty good, but I would have liked to seen be...",pretty good but i would have liked to seen bet...,"[pretty, good, but, i, would, have, liked, to,...",False
2,3,So so. Casting could have been improved,so so casting could have been improved,"[so, so, casting, could, have, been, improved]",False
3,5,The most EPIC movie of the year! Casting was a...,the most epic movie of the year casting was aw...,"[the, most, epic, movie, of, the, year, castin...",True
4,4,Solid but I would have liked to see more of th...,solid but i would have liked to see more of th...,"[solid, but, i, would, have, liked, to, see, m...",False
5,5,THE BOMB!!!!!!!,the bomb,"[the, bomb]",False


### That's it! Great Job!